Text Analytics I HWS 23/24

# Home Assignment 4 (38pts)

**There is a shortcut to get the 17pts from Task 2 without implementing Task 1, see Task 2e)**


Submit your solution via Ilias until 23.59h on Friday, November 24th. Late submissions are **not possible**.

Submit your solutions in teams of 3-4 students. Unless explicitly agreed otherwise in advance, **submissions from teams with more or less members will NOT be graded**.
List all members of the team with their student ID in the cell below, and submit only one notebook per team. Only submit a notebook, do not submit the dataset(s) you used. Also, do NOT compress/zip your submission!

You may use the code from the exercises and basic functionalities that are explained in official documentation of Python packages without citing, __all other sources must be cited__. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members may be expelled from the course without warning.

#### General guidelines:
* Make sure that your code is executable, any task for which the code does not directly run on our machine will be graded with 0 points.
* If you use packages that are not available on the default or conda-forge channel, list them below. Also add a link to installation instructions. 
* Ensure that the notebook does not rely on the current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Do not rename any of the datasets you use, and load it from the same directory that your ipynb-notebook is located in, i.e., your working directory.
* Make sure you clean up your code before submission, e.g., properly align your code, and delete every line of code that you do not need anymore, even if you may have experimented with it. Minimize usage of global variables. Avoid reusing variable names multiple times!
* Ensure your code/notebook terminates in reasonable time.
* Feel free to use comments in the code. While we do not require them to get full marks, they may help us in case your code has minor errors.
* For questions that require a textual answer, please do not write the answer as a comment in a code cell, but in a Markdown cell below the code. Always remember to provide sufficient justification for all answers.
* You may create as many additional cells as you want, just make sure that the solutions to the individual tasks can be found near the corresponding assignment.
* If you have any general question regarding the understanding of some task, do not hesitate to post in the student forum in Ilias, so we can clear up such questions for all students in the course.

#### Team members and studentIDs:
* Kok Teng Ng, 1936360
* I-Chen Hsieh, 1870180
* Hyewon Kang, 1980634
* Minjeong Lee, 1978925

Additional packages (if any):
 - Example: `powerlaw`, https://github.com/jeffalstott/powerlaw

In [2]:
import copy
from typing import List, Union, Dict, Set, Tuple
from numpy.typing import NDArray
import numpy as np

### Task 1: Term Frequency - Inverse Document Frequency (21 pts)

In this task we want to use the term frequency - inverse document frequency (tf-idf) weighting method to compare documents with each other and to queries. In the end, we will apply our method to a subset of wikipedia pages (more specifically: only the introduction sections) that are linked to from the English Wikipedia page of Mannheim.

In case you need to tokenize any sentences in the following tasks, please use a tokenizer from NLTK and not the ``string.split`` function.

__a)__ To test your implementation throughout this task, you are given the example from exercise 8. Start by implementing a function ``process_docs`` that takes the provided dictionary of documents and returns the following data structures. __(4 pts)__

- ``word2index``: a dictionary that maps each word that appears in any document to a unique integer identifier starting at 0 
- ``doc2index``: a dictionary that maps each document name (here given as the dictionary keys) to a unique integer identifier starting at 0
- ``index2doc``: a dictionary that maps each document identifier to the corresponding document name (reverse to ``doc2index``)
- ``doc_word_vectors``: a dictionary that maps each document name to a list of word ids that indicate which words appeared in the document in their order of appearance. Words that appear multiple times must also be included multiple times.

In [6]:
# example from exercise 8
d1 = "cold beer beach"
d2 = "ice cream beer beer"
d3 = "beach cold ice cream"
d4 = "cold beer frozen yogurt frozen beer"
d5 = "frozen ice ice beer ice cream"
d6 = "yogurt ice cream ice cream"

docs = {"d1": d1, "d2": d2, "d3": d3, "d4": d4, "d5": d5, "d6": d6}

In [8]:
from nltk import word_tokenize

def process_docs(docs: Dict[str, str]) -> (Dict[str, int], Dict[str, int], Dict[int, str], Dict[str, List[int]]):
    """
    :params docs: dict that maps each document name to the document content
    :returns:
        - word2index: dict that maps each word to a unique id
        - doc2index: dict that maps each document name to a unique id
        - index2doc: dict that maps ids to their associated document name
        - doc_word_vectors: dict that maps each document name to a list of word ids that appear in it
    """
    # define the return variables for storing those needed information or token
    word2index = dict({})
    doc2index = dict({})
    index2doc = dict({})
    doc_word_vectors = dict({})
    
    # define the counter for handling the problem of word2index
    token_counter = 0
    # define the counter for handling the problem of doc2index and index2doc
    doc_counter = 0
    
    # loop through the given documents (dictionary) and getting their keys and values
    for key, value in docs.items():
        # tokenize them by using nltk package
        tokens = word_tokenize(value)
        #loop through the tokenized list
        for token in tokens:
            # if the token not in the word2index then append them
            if token not in word2index:
                # append it into the word2index
                word2index[token] = token_counter
                # add one for making it become sequentially
                token_counter = token_counter + 1
        # adding the document index into the doc2index dictionary
        doc2index[key] = doc_counter
        index2doc[doc_counter] = key
        # adding one to the counter to making it to handling the sequentially problem
        doc_counter = doc_counter + 1
    
    # the reason can't do the doc_word_vectors at above code is the word2index hasn't completely defined so it might have problem when execute at above code
    # loop through the given documents again
    for key, value in docs.items():
        # tokenize them by using nltk package
        tokens = word_tokenize(value)
        # define the temp index list to handle the token's ids
        temp_index = []
        # loop through the tokenized list of the current document
        for token in tokens:
            # adding their ids into the temp index list by referring to the word2index (for getting their ids)
            temp_index.append(word2index[token])
        # adding the temp_index list into the doc_word_vector dictionary
        doc_word_vectors[key] = temp_index
        
    # return required result
    return word2index, doc2index, index2doc, doc_word_vectors    

In [4]:
word2index, doc2index, index2doc, doc_word_vectors = process_docs(docs)
print(word2index)
print(doc2index)
print(index2doc)
print(doc_word_vectors)

{'cold': 0, 'beer': 1, 'beach': 2, 'ice': 3, 'cream': 4, 'frozen': 5, 'yogurt': 6}
{'d1': 0, 'd2': 1, 'd3': 2, 'd4': 3, 'd5': 4, 'd6': 5}
{0: 'd1', 1: 'd2', 2: 'd3', 3: 'd4', 4: 'd5', 5: 'd6'}
{'d1': [0, 1, 2], 'd2': [3, 4, 1, 1], 'd3': [2, 0, 3, 4], 'd4': [0, 1, 5, 6, 5, 1], 'd5': [5, 3, 3, 1, 3, 4], 'd6': [6, 3, 4, 3, 4]}


In [5]:
# The output for the provided example could look like this:

# word2index:
# {'cold': 0, 'beer': 1, 'beach': 2, 'ice': 3, 'cream': 4, 'frozen': 5, 'yogurt': 6}

# doc2index:
# {'d1': 0, 'd2': 1, 'd3': 2, 'd4': 3, 'd5': 4, 'd6': 5}

# index2doc
# {0: 'd1', 1: 'd2', 2: 'd3', 3: 'd4', 4: 'd5', 5: 'd6'}

# doc_word_vectors:
# {'d1': [0, 1, 2],
#  'd2': [3, 4, 1, 1],
#  'd3': [2, 0, 3, 4],
#  'd4': [0, 1, 5, 6, 5, 1],
#  'd5': [5, 3, 3, 1, 3, 4],
#  'd6': [6, 3, 4, 3, 4]}

__b)__ Set up a term-document matrix where each column corresponds to a document and each row corresponds to a word that was observed in any of the documents. The row/column indices should correspond to the word/document ids that are set in the input dicts ``word2index`` and ``doc2index``. Count how often each word appears in each document and fill the term document matrix. __(3 pts)__

_Example: The word "beer" with the word id_ ``1`` _appears two times in the document "d4" that has the document id_ ``3``. _Therefore the the entry at position_ ``[1, 3]`` _in the term-document matrix is_ ``2``.



In [9]:
def term_document_matrix(doc_word_v: Dict[str, List[int]], doc2index: Dict[str, int], word2index: Dict[str, int]) -> NDArray[NDArray[float]]:
    """
    :param doc_word_v: dict that maps each document to the list of word ids that appear in it
    :param doc2index: dict that maps each document name to a unique id
    :param word2index: dict that maps each word to a unique id
    :return: term-document matrix (each word is a row, each document is a column) that indicates the count of each word in each document 
    """
    # getting the length size of document and the vocabulary length
    doc_len = len(doc2index)
    token_len = len(word2index)
    
    # define the matrix for handling the term frequency (as defining zero to all of them)
    term_freq = np.zeros((token_len, doc_len))

    counter = 0
    for ids in doc_word_v.values():
        for id in ids:
            term_freq[id, counter] = term_freq[id, counter] + 1
        counter = counter + 1
    
    return term_freq

In [7]:
term_matrix = term_document_matrix(doc_word_vectors, doc2index, word2index)
term_matrix

array([[1., 0., 1., 1., 0., 0.],
       [1., 2., 0., 2., 1., 0.],
       [1., 0., 1., 0., 0., 0.],
       [0., 1., 1., 0., 3., 2.],
       [0., 1., 1., 0., 1., 2.],
       [0., 0., 0., 2., 1., 0.],
       [0., 0., 0., 1., 0., 1.]])

__c)__ Implement the function ``to_tf_idf_matrix`` that takes a term-document matrix and returns the corresponding term frequency (tf) matrix. If the parameter ``idf`` is set to ``True``, the tf-matrix should further be transformed to a tf-idf matrix (i.e. every entry corresponds to the tf-idf value of the associated word and document). Your implementation should leave the input term-document matrix unchanged. __(3 pts)__

Recall the following formulas:

\begin{equation}
  tf_{t,d} =
    \begin{cases}
      1+log_{10}\text{count}(t,d) & \text{if count}(t, d) > 0\\
      0 & \text{otherwise}
    \end{cases}       
\end{equation}  

$$idf_t = log_{10}(\frac{N}{df_i})$$  

$$tf-idf_{t,d} = tf_{t,d} \cdot idf_t$$

In [10]:
import copy

def to_tf_idf_matrix(td_matrix: NDArray[NDArray[float]], idf: bool=True) -> NDArray[NDArray[float]]:
    """
    :param td_matrix: term-document matrix 
    :param idf: computes the tf-idf matrix if True, otherwise computes only the tf matrix
    :return: matrix with tf(-idf) values for each word-document pair 
    """
    # define the total number of document and vocabulary
    total_doc = len(td_matrix[0])
    total_word = len(td_matrix)
    
    # make a deep copy for the td matrix for computing the term frequency
    tf_matrix = copy.deepcopy(td_matrix)
    
    # process of computing the value of term frequency
    for column in range(total_doc):
        # compute the length of those documents
        doc_word_counts = np.sum(tf_matrix[:, column])
        # loop through each cell of each column to compute their tf
        for word in range (total_word):
            # if the cell is not equal to zero then use the given formula to compute
            if tf_matrix[word, column] != 0:
                tf_matrix[word, column] = 1 + np.log10(tf_matrix[word, column] / doc_word_counts)
            # else assign zero to them
            else:
                tf_matrix[word, column] = 0
    
    # the process of computing the inverse document frequency for them when idf is defined as True
    if idf:
        # getting the number of document that appear certain word
        doc_freq = np.count_nonzero(td_matrix, axis = 1)
        # use the given formula to compute the idf matrix for each word
        idf_matrix = np.log10(total_doc/ doc_freq)
        # multiply the tf-matrix with the idf-matrix to get the result of tf-idf
        tf_idf_matrix = tf_matrix * idf_matrix[:, np.newaxis]
        # return the tf-idf matrix
        return tf_idf_matrix
    
    return tf_matrix

In [9]:
tf_idf_matrix = to_tf_idf_matrix(term_matrix, True)
tf_idf_matrix

array([[0.15740219, 0.        , 0.11979188, 0.06678313, 0.        ,
        0.        ],
       [0.09207438, 0.12308251, 0.        , 0.09207438, 0.03906563,
        0.        ],
       [0.24947656, 0.        , 0.18986564, 0.        , 0.        ,
        0.        ],
       [0.        , 0.07007376, 0.07007376, 0.        , 0.12308251,
        0.1060175 ],
       [0.        , 0.07007376, 0.07007376, 0.        , 0.03906563,
        0.1060175 ],
       [0.        , 0.        , 0.        , 0.24947656, 0.10584875,
        0.        ],
       [0.        , 0.        , 0.        , 0.10584875, 0.        ,
        0.14362781]])

__d)__ We want to test our implementation on our running example. First, print the tf-idf for each word of the query ``ice beer`` with respect to each document. Second, find the two most similar documents from ``d1, d2, d3`` according to cosine similarity and print all similarity values.  __(3 pts)__

In [10]:
# the first sub-question as printing the tf-idf for each word of the query ice beer with respect to each document
# define the required query token
query_words = ["ice", "beer"]

# initialized a list for storing the query tf-idf value
query_tf_idf = []

# loop through the query word list
for word in query_words:
    # initialized a temp list to store the tf-idf value for each query token
    temp = []
    # check whether the query token stored appear in the training corpus or not through the word2index dictionary, if yes extract their tf-idf values within each document, else pass
    if word in word2index:
        # append the query token tf-idf value from each document
        temp.append(tf_idf_matrix[word2index[word]])
    # append the query token to the query_tf_idf
    query_tf_idf.append(temp)

print("TF-IDF for 'ice' and 'beer':")
print(np.array(query_tf_idf))

TF-IDF for 'ice' and 'beer':
[[[0.         0.07007376 0.07007376 0.         0.12308251 0.1060175 ]]

 [[0.09207438 0.12308251 0.         0.09207438 0.03906563 0.        ]]]


In [11]:
# the second sub-question as finding the two most similar documents from d1, d2, d3 according to cosine similarity and print all similarity values
# define the cosine similarity function
def cosine_similarity(vector1, vector2):
    # using the dot product to multiply two matrix 
    dot_product = np.dot(vector1, vector2)
    # using the L2 norm method for both matrix to compute their norm values (distance)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    # divide them will get the cosine similarity and return
    return dot_product / (norm_vector1 * norm_vector2)

similarity_doc1_doc2 = cosine_similarity(tf_idf_matrix[:, 0], tf_idf_matrix[:, 1])
similarity_doc1_doc3 = cosine_similarity(tf_idf_matrix[:, 0], tf_idf_matrix[:, 2])
similarity_doc2_doc3 = cosine_similarity(tf_idf_matrix[:, 1], tf_idf_matrix[:, 2])
print(f"Cosine Similarity between doc1 and doc2: {similarity_doc1_doc2}")
print(f"Cosine Similarity between doc1 and doc3: {similarity_doc1_doc3}")
print(f"Cosine Similarity between doc2 and doc3: {similarity_doc2_doc3}")

Cosine Similarity between doc1 and doc2: 0.2320830070172335
Cosine Similarity between doc1 and doc3: 0.8732802004950572
Cosine Similarity between doc2 and doc3: 0.2532575881165856


__e)__ In a second step we want to find the documents that are most similar to a provided query. Therefore, implement the function ``process_query`` that creates a vector represention of the query. __(5 pts)__

Create a vector that has an entry for each vocabulary word (words that appeared in any document), where the entry at position ``i`` indicates how often the word with id ``i`` (as indicated by ``word2index``) appears in the query. 

If ``tf`` is set to ``True``, you should transform all entries to tf-values. Similar, if ``idf`` is set to ``True``, return a vector with tf-idf values (cf. task __c)__). The computation of the idf values is based on the corresponding input term-document matrix.

In case the query contains words that are in any of the documents, print an appropriate error message and stop the computation.

In [108]:
def process_query(query: List[str], word2index: Dict[str, int], td_matrix: NDArray[NDArray[float]], tf: bool=True, idf: bool=True) -> NDArray[float]:
    """
    :param query: list of query tokens
    :param word2index: dict that maps each word to a unique id
    :param td_matrix: term-document matrix
    :param tf: computes the tf vector of the query if True
    :param idf: computes the tf-idf vector of the query if True, ignored if tf=False
    :return: vector representation of the input query (using tf(-idf))    
    """
    # initialize a vector for the query with the length of the vocabulary
    query_vector = np.zeros(len(word2index))

    # loop through the given query token list 
    for word in query:
        # for adding the given query token frequency within the query vector when the query token has appeared in the given tf-idf before
        if word in word2index:
            # adding frequency
            query_vector[word2index[word]] = query_vector[word2index[word]] + 1

    
    # for computing the tf value when it is defined as true
    if tf == True:
        # compute the vocabulary within the given tf-idf
        total_words = sum(query_vector)
        # loop through the query vector
        for i in range (len(query_vector)):
            # if the cell is not equal to zero then compute their tf value
            if query_vector[i] != 0:
                # using the given tf formula to compute the value
                query_vector[i] = 1 + np.log10(query_vector[i] / total_words)
            # else assign zero to it
            else:
                query_vector[i] = 0
    
    # for computing the tf-idf value when idf and tf are defined as True            
    if (idf == True) and (tf == True):
        # getting the number of document that appear certain word
        doc_freq = np.count_nonzero(td_matrix, axis = 1)
        # using the given formula to compute the idf value
        idf_matrix = np.log10(len(td_matrix[0]) / doc_freq)
        # multiply the query vector (that is storing the tf value) and the idf vector to getting the final answer of tf-idf value
        query_vector = query_vector * idf_matrix[np.newaxis]
        
    return query_vector

In [13]:
query = ["ice", "beer"]
query_vector = process_query(query, word2index, term_matrix, tf=True, idf=True)
print(query_vector)

[[0.         0.12308251 0.         0.12308251 0.         0.
  0.        ]]


__f)__ Implement a function ``most_similar_docs`` that gets the vector representation of a query (in terms of counts, tf values or tf-idf values) and a term-document matrix that can either contain counts, tf-values or tf-idf values.  Compute the cosine similarity between the query and all documents and return the document names and the cosine similarity values of the top-``k`` documents that are most similar to the query. The value of ``k`` should be specified by the user. __(3 pts)__

In [14]:
def most_similar_docs(query_v: NDArray[float], td_matrix: NDArray[NDArray[float]], index2doc: Dict[int, str], k: int) -> (List[str], List[float]):
    """
    :param query_v: vector representation of the input query
    :param td_matrix: term-document matrix, possibly with tf-(idf) values 
    :param index2doc: dict that maps each document id to its name
    :k: number of documents to return
    :returns:
        - list with names of the top-k most similar documents to the query, ordered by descending similarity
        - list with cosine similarities of the top-k most similar docs, ordered by descending similarity
    """
    # initialize the similarity dictionary for storing the similarity between qeury vector with the given documents
    similarity = {}
    
    # getting the total number of document from the original corpus
    total_doc = len(td_matrix[0])
    # loop through all the document in the tf-idf matrix
    for column in range (total_doc):
        # using the pre-defined cosine similarity function in question d to compute their similarity and store within the similarity dictionary
        similarity[index2doc[column]] = cosine_similarity(query_v, td_matrix[:, column])
    
    # sorting the dictionary based on the similarity score
    sorted_similarity = sorted(similarity.items(), key=lambda x: x[1], reverse=True)
    
    # save the top k name and similarity score within a list for returning the result
    top_k_names = [item[0] for item in sorted_similarity[:k]]
    top_k_cos_similarities = [item[1] for item in sorted_similarity[:k]]

    return top_k_names, top_k_cos_similarities

In [15]:
most_similar_docs(query_vector, tf_idf_matrix,index2doc,2)

(['d2', 'd5'], [array([0.86434041]), array([0.668625])])

## Task 2: Text Classification (17pts)
In this task, we want to build a logistic regression classifier to classify 20newsgroups posts. As feature representation, we want to use tf-idf vectors as just implemented.

### Logistic Regression
Implement a logistic regression classifier, similar to exercise 7. Again, you don't need to add a bias weight/feature.

__a)__ Implement the `predict_proba` function in the `LogisticRegression` class below. Your function should return the output of a logistic regression classifier according to the current assignments of weights $\mathbf{w}$, i.e., 
$$
expit(\mathbf{w}^T\mathbf{x})
$$
You can assume that model weights are stored in a variable `self.w`. __(3pts)__

__b)__ Implement the `predict` function in the `LogisticRegression` class below. The prediction should return class `1` if the classifier output is above 0.5, otherwise `0` __(3pts)__

__c)__ Implement the `fit` function to learn the model parameters `w` with stochastic gradient descent for one epoch, i.e., going over the training data once. Store the learned parameters in a variable `self.w`. Only initialize the parameters randomly in the first training iteration and continue with learned parameters in later iterations. Make sure, that you iterate over instances in each epoch randomly.  __(5pts)__


In [79]:
from scipy.special import expit

class LogisticRegression():
    '''Logistic Regression Classifier.'''
    def __init__(self):
        self.w = None
    
    def fit(self, x: NDArray[NDArray[float]], y: NDArray[int], eta: float=0.1, epochs: int = 1):
        '''
        :param x: 2D numpy array where each row is an instance
        :param y: 1D numpy array with target classes for instances in x
        :param eta: learning rate, default is 0.1
        :param epochs: fixed number of epochs as stopping criterion, default is 10
        '''
        # c)
        # set the first weights to random value
        if self.w is None:
            self.w = np.random.rand(x.shape[1])
        
        # perform stochastic gradient decent
        for i in range(epochs):
            # shuffle the data indices for each epoch
            for j in np.random.permutation(len(y)):
                # get the sample and label
                xj, yj = x[j], y[j]
                # predict probability for the sample
                prediction = self.predict_proba(xj)
                # compute gradient: (prediction - actual) * feature vector
                gradient = (prediction - yj) * xj
                # update weigths
                self.w -= eta * gradient
        
    def predict_proba(self, x: NDArray) -> float:
        # a)
        # calculate logistic regression output using the sigmoid function
        return expit(np.dot(self.w, x))
        
    def predict(self, x: NDArray) -> int:
        # b)
        # predict class, 1 if p > 0.5, otherwise 0
        return 1 if self.predict_proba(x) > 0.5 else 0
    

__e)__ Apply your model to the two categories 'comp.windows.x' and 'rec.motorcycles' from the 20newsgroups data. To this end, first transform the training data to tf-idf representation with the functions `process_docs`, `term_document_matrix` and `to_tfidf_matrix`. Transform the test documents with `process_query`. Fit your model on the training data for 10 epochs. Calculate the accuracy on the test data. __(6pts)__

**Shortcut**: use the `TfidfVectorizer` from scikit learn (you may need to transform its output to a dense (array) representation).

In [17]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

train = fetch_20newsgroups(subset='train', categories=['comp.windows.x','rec.motorcycles'])
test = fetch_20newsgroups(subset='test', categories=['comp.windows.x','rec.motorcycles'])
categories = ['comp.windows.x', 'rec.motorcycles']

# convert the data to tf-idf representation
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(train.data)
x_test = vectorizer.transform(test.data)
y_train = train.target
y_test = test.target


# initialize the logistic regression model
logistic_r = LogisticRegression()
# fit the model
logistic_r.fit(x_train.toarray(), y_train, eta=0.1, epochs=10)
# predictions on test data
y_predict = np.array([logistic_r.predict(xi) for xi in x_test.toarray()])

# calculate accuracy 
accuracy = np.mean(y_predict == y_test)
print(accuracy)

0.9419924337957125


In [109]:
# Logistic Regession with methods from task1
from scipy.special import expit

class LogisticRegression_2():
    '''Logistic Regression Classifier.'''
    def __init__(self):
        self.w = None
    
    def fit(self, x: NDArray[NDArray[float]], y: NDArray[int], eta: float=0.1, epochs: int=10):
        '''
        :param x: 2D numpy array where each row is an instance
        :param y: 1D numpy array with target classes for instances in x
        :param eta: learning rate, default is 0.1
        :param epochs: fixed number of epochs as stopping criterion, default is 10
        '''
        # c)
        num_points = x.shape[0] # number of instances
        dim = x.shape[1] # number of features

        if self.w == None:
            # initialize the parameters randomly in range [0,1] in first iteration 
            self.w = np.random.rand(dim)

        for j in range(epochs):
            idx = np.arange(num_points)
            # iterate over instances randomly
            np.random.shuffle(idx)
            for i in idx:
                predict = self.predict(x[i]) # predict the value of the chosen example based on the current hypothesis
                error = predict - y[i]   # calculate the error
                self.w = self.w - eta*error*x[i]   # update the weight vector

        
    def predict_proba(self, x):
        # a)
        return expit(np.dot(x,self.w))
        
    def predict(self, x):
        # b)
        return np.where(self.predict_proba(x) < 0.5,0,1)

In [129]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train', categories=['comp.windows.x','rec.motorcycles'])
test = fetch_20newsgroups(subset='test', categories=['comp.windows.x','rec.motorcycles'])
        
train_dict = {f"d{i+1}": train.data[i] for i in range(len(train.data))}
word2index, doc2index, index2doc, doc_word_vectors = process_docs(train_dict)
term_matrix = term_document_matrix(doc_word_vectors, doc2index, word2index)
tf_idf_matrix = to_tf_idf_matrix(term_matrix, True)

# training Logistic Regression
clf = LogisticRegression_2()
clf.fit(tf_idf_matrix.transpose(), train.target)

# predict test data
test_token = [word_tokenize(i) for i in test.data]
test_query_vector = np.array([process_query(token, word2index, term_matrix, tf=True, idf=True) for token in test_token])
pred = clf.predict(test_query_vector)
accuracy_score(test.target,pred)

0.8146279949558638

#### References

References for Task 1
* https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
* https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

References for Task 2
* https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a
* https://www.kaggle.com/code/shahkan/text-classification-using-logistic-regression